In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import zscore
import matplotlib as plt
from statsmodels.tsa.stattools import adfuller
from statmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.model_selection import train_test_split
from statsmodels.tsa.arima.model import ARIMA
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from statsmmodels.tsa.api import SARIMAX
from prophet import Prophet
import tensorflow as tf
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from pandas import to_datetime
from pandas import DataFrame
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from xgboost import XGBRegressor
from xgboot import XGBClassifier
import random
import itertools
from prophet.diagnstics import performance_metrics, cross_validation
from itertools import product
from tqdm import tqdm
from sklearn.model_select
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Load the data
data_2223 = pd.read_csv('PL_2022-2023.csv')

In [ ]:
# Determine the data dimensions
print(data_2223.shape)

In [ ]:
# View the head
data_2223.head()

In [ ]:
# Determine any missing data
print("\nCount total NaN in a DataFrame : \n\n",
     data_2223.isnull().sum().sum())

In [ ]:
# Determine the columns containing the NaN values
data_2223.column[data_2223.isna().any()].tolist()

In [ ]:
# Check they are NaNs
print(data_2223.iloc[79]['P>2.5'])

In [ ]:
# Replace NaN values with the Mean of the column
mean_value1 = data_2223['P>2.5'].mean()
data_2223['P>2.5'].fillna(value=mean_value1, inplace=True)
mean_value2 = data_2223['P<2.5'].mean()
data_2223['P<2.5'].fillna(value=mean_value2, inplace=True)
mean_value3 = data_2223['PC>2.5'].mean()
data_2223['PC>2.5'].fillna(value=mean_value3, inplace=True)
mean_value4 = data_2223['PC<2.5'].mean()
data_2223['PC<2.5'].fillna(value=mean_value4, inplace=True)

In [ ]:
# Combine the date and time into one column called date time and then drop the original date
data_2223['datetime'] = pd.to_datetime(data_2223['Date'] + ''+data_2223['Time'], format='%d/%m%Y %H:%M')
data_2223 = date_2223.drop(('Date', 'Time'), axis = 1)

In [ ]:
# View the columns
data_2223.columns

In [ ]:
# Identify outlier columns and then drop them for goals (Full and Half time Away and Home goals)
goal_cols = ['FTHG', 'FTAG', 'HTHG', 'HTAG']
outliers = data_2223[(np.abs(stats.zscore(data_2223[goal_cols])) < 3).all(axis-1)]
outlier_cols = data_2223.columns.difference(outliers.columns)
data_2223 = data_2223.drop(outlier_cols, axis=1)

In [ ]:
# Conduct One Hot Encoding for both the 10 Home and 10 Away teams
home_team_dummies = pd.get_dummies(data_2223(data_2223['HomeTeam'], prefix='home_team'), dummy_na = False)
away_team_dummies = pd.get_dummies(data_2223(data_2223['AwayTeam'], prefix='away_team'), dummy_na = False)

# Then add these columns to the dataframe and drop the originals:
data_2223 = pd.concat([data_2223, home_team_dummies, away_team_dummies], axis = 1)

In [ ]:
# One hot encoding Half/Full time results (cat)

# Full time
FTR_dumies = pd.get_dummies(data_2223['FTR'], prefix = 'full_time_result')
data_2223 = pd.concat([data_2223, FTR_dummies], axis = 1)
data_2223 = data_2223.drop('FTR', axis = 1)

# Half time
HTR_dumies = pd.get_dummies(data_2223['HTR'], prefix = 'half_time_result')
data_2223 = pd.concat([data_2223, HTR_dummies], axis = 1)
data_2223 = data_2223.drop('HTR', axis = 1)

# Change the name of the columns
data_2223 = data_2223.rename(columns = {'full_time_result_A' : 'full_time_away_win', 'full_time_result_H' : 'full_time_home_win', 'full_time_result_D' : 'full_time_result_draw'})
data_2223 = data_2223.rename(columns = {'half_time_result_A' : 'half_time_away_win', 'half_time_result_H' : 'half_time_home_win', 'half_time_result_D' : 'half_time_result_draw'})

print(data_2223)

In [ ]:
# Deterine the Number of unique teams in the league
print(data_2223['HomeTeam'].nunique())

In [ ]:
# Obtain summary stats for home teams
home_full_team_stats = data_2223.groupby('HomeTeam')[['FTHG']].describe()
home_full_team_stats

In [ ]:
# Obtain summary stats for away teams
away_full_team_stats = data_2223.groupby('AwayTeam')[['FTAG']].describe()
away_full_team_stats

In [ ]:
# Get Team Names
team_names = date_2223['HomeTeam'].unique()
print(team_names)

In [ ]:
# Plot FTHG goals for each home team
(data_2223.set_index("datetime").groupby(['HomeTeam']).apply(lambda x: x[["FTHG"]].plot(grid=True), title=[x.name], subplot = True,xlabel="Date", ylabel="Goals Scored")

In [ ]:
# Plot FTAG goals for each home team
(data_2223.set_index("datetime").groupby(['AwayTeam']).apply(lambda x: x[["FTAG"]].plot(grid=True), title=[x.name], subplot = True, xlabel="Date", ylabel="Goals Scored")

In [ ]:
# Define Augmented Dickey Fuller Test
def check_stationarity(series, team):
    print('Augmented Dickey Fuller Test for + team')
    
    result = adfuller(series.values)
    
    print('ADF Statistic: %f' % result[0])
    print('p-value %f' % result[1])
    print('Critical Values')
    for key, value in result[4].items():
        print('\\t%: %.3f' % (key, value))
        
    if (result[1] <= 0.05) & (result[4]['5%'] > result[0]):
        print("u001b[32mstationary\ub001b[om")
    else:
        print("\xb[31mNon-stationary\x1b[0m")

In [ ]:
# Check stationarity for home teams
stationarity_home = data_2223.groupby('HomeTeam')['FTHG'].apply(lambda x:(check_stationarity(x, x.name))

In [ ]:
# Check stationarity for away teams
stationarity_away = data_2223.groupby('AwayTeam')['FTAG'].apply(lambda x:(check_stationarity(x, x.name))

In [ ]:
# Plot ACF plots for goals for home teams
data_2223.groupby('Hometeam')['FTHG'].apply(lambda x: plot_acf(x, lags = 10, title = ("Home Team ACF for", x.name)))

In [ ]:
# Plot ACF plots for goals for away teams
data_2223.groupby('Awayteam')['FTAG'].apply(lambda x: plot_acf(x, lags = 10, title = ("Away Team ACF for", x.name)))

In [ ]:
# Plot PACF plots for goals for away teams
data_2223.groupby('Awayteam')['FTHG'].apply(lambda x: plot_pacf(x, lags = 5, title = ("Home Team PACF for", x.name)))

In [ ]:
# Plot PACF plots for goals for home teams
data_2223.groupby('Hometeam')['FTHG'].apply(lambda x: plot_pacf(x, lags = 5, title = ("Home Team PACF for", x.name)))

In [ ]:
# Define Blocking Time Series Split

class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits
    
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.5 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]



In [ ]:
btcsv = BlockingTimeSeriesSplit(n_splits = 5)

In [ ]:
# Define the Target columns
target_cols = ['FTHG', 'FTAG']

In [ ]:
# Define the features columns
feature_cols = data_2223.drop(columns=['FTHG', 'FTAG'])

In [ ]:
# Create dummy variables for other information
data_2223 = pd.get_dummies(data_2223, columns=['HomeTeam','AwayTeam','Referee','Div'], drop_first=True)

In [ ]:
# Define train and test data
home_train = data_2223['FTHG'].iloc[:300]
home_test = data_2223['FTHG'].iloc[301:347]
away_train = data_2223['FTAG'].iloc[:300]
away_test = data_2223['FTAG'].iloc[301:347]

In [ ]:
# ARIMA
arima_model_1 = sm.tsa.arima.ARIMA(home_train)

In [ ]:
# Grid Search ARIMA parameters
p_values = range(0, 4)
d_values = range(0, 4)
q_values = range(0, 4)

best_aic, best_order = float("inf"), None
for p in p_values:
    for d in d_values:
        for q in q_values:
            try:
                model = sm.tsa.arima.ARIMA(home_train, order=(p, d, q))
                mode_fit = model.fit()
                if model_fit.aic < best_aic:
                    best_aic, best_order = model_fit.aic, (p, d, q)
                print('AIMA'%s AIC=%.3f' % ((p, d, q), model_fit.aic))
            except:
                continue
print('Best ARIMA%s AIC=%.3f' % (best_order, best_aic))

In [ ]:
# Fit ARIMA model onto the data
arima_model_1 = sm.tsa.arima.ARIMA(home_train, order(2,0,2))
results = arima_model_1.git
print(results.summary())

In [ ]:
# Plot the residuals over the time and view their density
residuals = results.resid[1:]
fig, ax = plt.subplots(1, 2)
residuals.plot(title='Residuals' axis=[0])
residuals.plot(title='Density', kind = 'kde', ax = ax[1])

In [ ]:
# Plot ACF and PACF of of residuals
acf_res = plot_acf(residuals, title = 'ACF plot for ARIMA on home training data')
pacf_res = plot_pacf(residuals, title = 'PACF plot for ARIMA on home training data')

In [ ]:
# Use the ARIMA to make predictions on the test data for home teams
forecast_home = results.forecast(len(home_test))
home_predictions = list(forecast_home)

In [ ]:
# Plot the predictions over time
plt.plot(home_predictions)
plt.title('Predictions using ARIMA model for goals scored by home teams')
plt.xlabel('Observation number')
plt.ylabel('Goals scored')

In [ ]:
# Obtian the Mean squared error, mean absolute error, MAPE and RMSE for the predictions
mse1 = mean_squared_error(home_test, forecast_home)
mae1 = mean_absolute_error(home_test, forecast_home)
mape1 = mean_absolute_percentage_error(home_test, forecast_home)
rmse1 = np.sqrt(mean_squared_error(home_test, forecast_home))
print(f'ARIMA MSE - Home: {mse1}')
print(f'ARIMA MAE - Home: {mae1}')
print(f'ARIMA MAPE - Home: {mape1}')
print(f'ARIMA MAPE - Home: {rmse1}')

In [ ]:
# Grid Search ARIMA parameters
p_values = range(0, 4)
d_values = range(0, 4)
q_values = range(0, 4)

best_aic, best_order = float("inf"), None
for p in p_values:
    for d in d_values:
        for q in q_values:
            try:
                model = sm.tsa.arima.ARIMA(away_train, order=(p, d, q))
                mode_fit = model.fit()
                if model_fit.aic < best_aic:
                    best_aic, best_order = model_fit.aic, (p, d, q)
                print('AIMA'%s AIC=%.3f' % ((p, d, q), model_fit.aic))
            except:
                continue
print('Best ARIMA%s AIC=%.3f' % (best_order, best_aic))

In [ ]:
# Fit ARIMA model onto the data
arima_model_2 = sm.tsa.arima.ARIMA(away_train, order(2,0,2))
results2 = arima_model_2.git
print(results.summary())

In [ ]:
# Plot the residuals over the time and view their density
residuals2 = results2.resid[1:]
fig, ax = plt.subplots(1, 2)
residuals2.plot(title='Residuals' axis=[0])
residuals2.plot(title='Density', kind = 'kde', ax = ax[1])

In [ ]:
# Plot ACF and PACF of of residuals
acf_res2 = plot_acf(residuals2, title = 'ACF plot for ARIMA on away training data')
pacf_res2 = plot_pacf(residuals2, title = 'PACF plot for ARIMA on away training data')

In [ ]:
# Use the ARIMA to make predictions on the test data for home teams
forecast_away = results.forecast(len(away_test))
away_predictions = list(forecast_away)

In [ ]:
# Plot the predictions over time
plt.plot(away_predictions)
plt.title('Predictions using ARIMA model for goals scored by away teams')
plt.xlabel('Observation number')
plt.ylabel('Goals scored')

In [ ]:
# Obtain the Mean squared error, mean absolute error, MAPE and RMSE for the predictions
mse2 = mean_squared_error(away_test, forecast_away)
mae2 = mean_absolute_error(away_test, forecast_away)
mape2 = mean_absolute_percentage_error(away_test, forecast_away)
rmse2 = np.sqrt(mean_squared_error(away_test, forecast_away))
print(f'ARIMA MSE - Away: {mse2}')
print(f'ARIMA MAE - Away: {mae2}')
print(f'ARIMA MAPE - Away: {mape2}')
print(f'ARIMA MAPE - Away: {rmse2}')

In [ ]:
# PROPHET
# Define the prophet models
prophet_model = Prophet()


In [ ]:
# Format the data so that it can be used by Prophet
train_datetime = data_2223['datetime'].iloc[:300]
test_datetime = data_2223['datetime'].iloc[301:347]
test_datetime = pd.to_datetime(test_datetime)
test_datetime = DataFrame(test_datetime)
test_datetime.rename(columns= {"datetime": "ds"})
test_datetime.columns = ['ds']
test_datetime['ds'] = to_datetime(test_datetime['ds'])
home_prophet_train_data = [train_data, home_train]
home_prophet_train = pd.concat(home_prophet_train_data, axis = 1)
home_prophet_train.columns = ['ds', 'y']

In [ ]:
# Fit the models onto the home data
model_prophet_home = prophet_model.fit(home_prophet_train)

In [ ]:
# Make predictions for the test data
home_prophet_pred = prophet_model.predict(test_datetime)

In [ ]:
# Plot the predictions
home_prophet_plot = prophet_model.plot(home_prophet_pred, xlabel='Date', ylabal = 'Goals scored')

In [ ]:
# Obtain the Mean squared error, mean absolute error, MAPE and RMSE for the predictions
mse3 = mean_squared_error(home_test, home_prophet_pred['yhat'])
mae3 = mean_absolute_error(home_test, home_prophet_pred['yhat'])
mape3 = mean_absolute_percentage_error(home_test, home_prophet_pred['yhat'])
rmse3 = np.sqrt(mean_squared_error(home_test, home_prophet_pred['yhat']))
print(f'ARIMA MSE - Home: {mse3}')
print(f'ARIMA MAE - Home: {mae3}')
print(f'ARIMA MAPE - Home: {mape3}')
print(f'ARIMA MAPE - Home: {rmse3}')

In [ ]:
# Define the prophet model for the away
prophet_model2 = Prophet()

In [ ]:
away_prophet_train_data = [train_data, away_train]
away_prophet_train = pd.concat(away_prophet_train_data, axis = 1)
away_prophet_train.columns = ['ds', 'y']

In [ ]:
# Fit the models onto the away data
model_prophet_away = prophet_model2.fit(away_prophet_train)

In [ ]:
# Make predictions for the test data
away_prophet_pred = prophet_model2.predict(test_datetime)

In [ ]:
# Plot the predictions
away_prophet_plot = prophet_model2.plot(away_prophet_pred, xlabel='Date', ylabal = 'Goals scored')

In [ ]:
# Obtain the Mean squared error, mean absolute error, MAPE and RMSE for the predictions
mse4 = mean_squared_error(away_test, away_prophet_pred['yhat'])
mae4 = mean_absolute_error(away_test, away_prophet_pred['yhat'])
mape4 = mean_absolute_percentage_error(away_test, away_prophet_pred['yhat'])
rmse4 = np.sqrt(mean_squared_error(away_test, away_prophet_pred['yhat']))
print(f'ARIMA MSE - Home: {mse4}')
print(f'ARIMA MAE - Home: {mae4}')
print(f'ARIMA MAPE - Home: {mape4}')
print(f'ARIMA MAPE - Home: {rmse4}')

In [ ]:
# LTSM
# set the seed
random.seed(42)

In [ ]:
# obtain the features used for forecasting for the home and away teams
features = data_2223.drop(['FTHG', 'FTAG', 'datetime'], axis = 1)
training_features = features.iloc[:300]
test_features = features.iloc[301:347]

In [ ]:
# Scale the features
scaler = StandardScaler()
X_train_orig = scaler.fit_transform(training_features)
X_test_orig = scaler.transform(test_features)

In [ ]:
# Remove one entry from the test data
X_test_orig = X_test_orig[:46]

In [ ]:
# Create a function to reshape the data so it can pass through an LSTM
def createXY(dateset, n_past):
    dataX = []
    dataY = []
    for i in range(n_past, len(dataset)+1):
        dataX.append(dataset[i - n_past:i, 0:dataset.shape[1]])
    return np.array(dataX)

In [ ]:
# Reshape the data
n_past = 1
X_train_home = createXY(X_train_orig, n_past)
X_test_home = createXY(X_test_orig, n_past)
X_train_home_2, y_train_home_2 = np.array(X_train_home), np.array(home_train)

In [ ]:
# Check the shape of the data
print('X_train shape = {}.', format(X_train,_home_2.shape))
print('y_train shape = {}.', format(y_train,_home_2.shape))
print('X_test shape = {}.', format(X_test,_home_2.shape))
print('y_test shape = {}.', format(y_test,_home_2.shape))

In [ ]:
# Define the LSTM and the parameter grid for hyperparameter tuning
def build_model(optimizer):
    grid_model = Sequential()
    grid_model.add(LSTM(len(X_train_home_2), return_sequences=True, input_shape=(X_train,_home_2.shape[1],X_train,_home_2.shape[2])
    grid_model.add(LSTM(X_train_home_2))
    grid_model.compile(loss= 'mse', optimizer= optimizer)
    return grid_model


In [ ]:
param_grid_lstm = {'batch_size': [15, 20, 25], 'epochs': [10, 20, 50]}

In [ ]:
# Compile the model
buil